In [12]:
# Imports
import pandas as pd
import os
import pydicom
import numpy as np
import cv2
import omidb

DICOM to PNG Conversion

In [ ]:
# Read the csv file
contralateral_case = pd.read_csv('contralateral_case.csv')

# Drop the 'fov_type' column
contralateral_case = contralateral_case.drop('fov_type', axis=1)

# Add filename column
contralateral_case['filename'] = None

for idx, row in contralateral_case.iterrows():
    # Get image name
    file_path_splitted = row['path'].split('/')
    image_name = file_path_splitted[11]
    
    # Read DICOM image
    dicom_image = pydicom.dcmread(row['path'])
    
    # Extract image specifications
    window_center = float(dicom_image.WindowCenter)
    window_width = float(dicom_image.WindowWidth)
    rescale_intercept = int(dicom_image.RescaleIntercept)
    rescale_slope = int(dicom_image.RescaleSlope)
    
    # Rescale the image using slope and intercept
    img = dicom_image.pixel_array.astype(float) * rescale_slope + rescale_intercept

    # Apply the window center and width
    img_min = window_center - (window_width / 2.0)
    img_max = window_center + (window_width / 2.0)
    img[img < img_min] = img_min
    img[img > img_max] = img_max

    # Normalize pixel values to [0, 255]
    img = (img - img_min) / (img_max - img_min)
    img = (img * 255).astype(np.uint8)

    # Save the image as a PNG file with the same name as the DICOM image in the converted subfolder
    png_image_name = image_name.replace(".dcm", ".png")
    png_image_path = os.path.join('contralateral_images', png_image_name)
    if row['image_laterality'] == 'R':
        img = cv2.flip(img, 1)
    cv2.imwrite(png_image_path, img)
    contralateral_case.at[idx, 'filename'] = png_image_name
contralateral_case.to_csv("contralateral_case_with_filename.csv", index=False)

Crop Background

In [ ]:
reading_path = 'contralateral_images'
csv_path = 'contralateral_case_with_filename.csv'
output_path = 'cropped_contralateral_images'

In [ ]:
def get_normal_BBox(image):
    img = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)[1]
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
    img2 = np.zeros(output.shape,dtype=np.uint8)
    img2[output == max_label] = 255
    contours, hierarchy = cv2.findContours(img2,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnt = contours[0]
    aux_im = img2
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(aux_im,(x,y),(x+w,y+h),(255,0,0),5)
    out_bbox = omidb.mark.BoundingBox(x, y, x+w, y+h)
    
    return out_bbox, img2 # returns bounding box and mask image.

In [ ]:
def crop_images(image, aux_folder):
    bbox,mask = get_normal_BBox(image)
    image_crop = image[bbox.y1:bbox.y2,bbox.x1:bbox.x2]
    cv2.imwrite(aux_folder,image_crop)
    
    return bbox

In [ ]:
df_contralateral = pd.read_csv(csv_path)
df_contralateral['bbox'] = None
for idx, row in df_contralateral.iterrows():
        output_image_path = os.path.join(output_path, row['filename'])

        filename = os.path.join(reading_path, row['filename'])
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        image = np.uint8(img)
        crop_bbox = crop_images(image, output_image_path)
        df_contralateral.at[idx, 'bbox'] = crop_bbox
df_contralateral.to_csv("cropped_contralateral_case.csv",index=False)